In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer

In [15]:
# change it to your df address
df = pd.read_pickle('C:/Users/miaoy/Desktop/11785/HwData/project/df_246_samples_pca_a.pkl')

In [17]:
df

,investment_id,time_id,fpca_0,fpca_1,fpca_2,fpca_3,fpca_4,fpca_5,fpca_6,fpca_7,...,fpca_103,fpca_104,fpca_105,fpca_106,fpca_107,fpca_108,fpca_109,fpca_110,fpca_111,target
0,16,401,4.125689,-0.087456,-0.751491,2.636128,-0.342277,-1.242430,-1.171127,0.601581,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,-0.714915
51597,941,401,-1.424219,0.629828,0.619910,1.693061,0.579582,0.620875,1.013102,-0.498025,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.381396
179361,3352,401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
50778,939,401,3.584108,-3.634416,2.589416,0.146125,-2.431967,-0.328496,-2.160348,0.225488,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.567380
49959,919,401,7.256094,-0.455397,1.040221,-2.464299,0.946264,0.368885,-3.076227,-0.806560,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.399060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129401,2332,1219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
128582,2329,1219,0.139191,0.663212,1.647383,0.726126,1.144654,-0.102302,0.826424,-1.920998,...,0.258343,-0.016063,-0.375717,-0.150204,0.0,0.0,0.0,0.0,0.0,0.228585
127763,2325,1219,0.631267,1.170344,-0.696518,-0.704307,0.660254,-0.115331,0.776461,0.278323,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.070355
126125,2245,1219,-4.267131,0.530890,-3.734727,2.001782,0.488225,0.063328,-0.463631,0.662666,...,0.801770,-0.866872,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,-0.019058


In [21]:
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=False)
df_train, df_val = train_test_split(df_train, test_size=2/9, shuffle=False)
max_prediction_length=3
max_encoder_length=14

train_dataset = TimeSeriesDataSet(
    df_train,
    group_ids=["investment_id"],
    target="target",
    time_idx="time_id",
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["investment_id"],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=df.columns.tolist()[2:-1],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=['target'],
    target_normalizer=GroupNormalizer( # normalize the targe for each investment_id along corresponding time_idx
        groups=["investment_id"], 
        transformation=None # NOTE: do not use softplus or relu for encoder normalization with DeepAR
        # transformation="softplus" # NOTE: do not use softplus or relu for encoder normalization with DeepAR
    ),
    # Add additional features
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)
val_dataset = TimeSeriesDataSet.from_dataset(train_dataset, df_val, predict=True, stop_randomization=True)
test_dataset = TimeSeriesDataSet.from_dataset(train_dataset, df_test, predict=True, stop_randomization=True)
# Save dataset to accelerate
train_dataset.save('C:/Users/miaoy/Desktop/11785/HwData/project/pf_train_246_samples_arpca.pf')
val_dataset.save('C:/Users/miaoy/Desktop/11785/HwData/project/pf_val_246_samples_arpca.pf')
test_dataset.save('C:/Users/miaoy/Desktop/11785/HwData/project/pf_test_246_samples_arpca.pf')